# Consommation des moteurs.

## Etape 1. Récupération et sauvegarde de la base d'indicateurs.

In [45]:
import os
from glob import glob
import numpy as np
import pandas as pd
from tabata import Opset

%reload_ext autoreload
%autoreload 2

In [46]:
datadir = "../data/"
datafiles = glob(os.path.join(datadir,'*.h5'))
sorting = lambda name: int(name[8:])

In [47]:
phaselist = ['climb', 'cruise', 'descend']

In [48]:
def detect_phase_local(df, threshold):
    '''
        Extracting rules:
        df['M [Mach]'] == 0 => taxi
        df['ALT [ft]] == threshold * max altitude => cruise (because using alt is more stable than using Mach)
        other = climb/descend

        Input: 
            df: flight dataframe
            look: 
            threshold: threshold for extracting cruising phase
        Output:
            Fuel consumption of specified phase

    '''
    #Get taxi idx
    #There are 2 taxi phases
    mc = df['M [Mach]']
    cruise_pos = df['ALT [ft]'].idxmax() 
    #Last time I took from the middle of the recording resulting in wrong extraction 
    # of some flights that has very long first taxi phase
    idx = mc [ mc == 0 ].index # both phases indices
    idx1 = idx[idx < cruise_pos] #one taxi phase before cruise
    idx2 = idx[idx > cruise_pos] #another one after cruise
    
    if len(idx1) == 0 or len(idx2) == 0: #If the record is incomplete
        print('    Record is incomplete', end=' ')
        return False
    else:
        taxi1_idx = min(idx1), max(idx1)
        taxi2_idx = min(idx2), max(idx2)
    
    #Get cruise idx
    # cruise_idx = extend(cruise_pos,df['ALT [ft]'], look, threshold)
    a = df['ALT [ft]']
    idx = a [ ( a > max(a)*(1 - threshold) ) ].index
    cruise_idx = min(idx), max(idx)

    #Get climb idx
    climb_idx = taxi1_idx[1], cruise_idx[0]
    #Get descend idx
    descend_idx = cruise_idx[1], taxi2_idx[0]

    # On supprime les vols mal découpés.
    if np.diff(climb_idx)<=1:
        print('    Climb is not found', end=' ')
        return False
    if np.diff(cruise_idx)<=1:
        print('    Cruise is not found', end=' ')
        return False
    if np.diff(descend_idx)<=1:
        print('    Descend is not found', end=' ')
        return False

    return taxi1_idx, climb_idx, cruise_idx, descend_idx, taxi2_idx 

def get_consumption_local(ac, phase: str = None, threshold: int = 0.05):
    '''
        Input: 
            ac: .h5 files of flight recordings
            phase: None (the whole flight) or one of 'taxi','taxi1', 'taxi2', 'climb','cruise','descend'
            altitude_threshold: threshold for extracting cruising phase 
        Output:
            Fuel consumption of specified phase
    '''

    dat = []
    for i, df in enumerate(ac):

        if len(df.columns)>0 and "ALT [ft]" in df.columns:
            
            alt = df["ALT [ft]"]
            Alt_max = max(alt)
            

            if Alt_max > 20000: # Maxmimum altitude can indicates if a recording is meaningful or not
                
                ## selecting phase index
                phases = detect_phase_local(df, threshold) 
                # phases = taxi1_idx, climb_idx, cruise_idx, descend_idx, taxi2_idx

                if phases == False:
                    print(f'  flight {i}')
                    continue
                else:
                    taxi1_idx, climb_idx, cruise_idx, descend_idx, taxi2_idx = phases

                if phase == 'taxi':
                    idx = np.concatenate([np.arange(taxi1_idx[0],taxi1_idx[1]),
                                          np.arange(taxi2_idx[0],taxi2_idx[1])])
                elif phase == 'taxi1':
                    idx = np.arange(taxi1_idx[0],taxi1_idx[1])
                elif phase == 'taxi2':
                    idx = np.arange(taxi2_idx[0],taxi2_idx[1])
                elif phase == 'climb':
                    idx = np.arange(climb_idx[0],climb_idx[1])
                elif phase == 'cruise':
                    idx = np.arange(cruise_idx[0],cruise_idx[1])
                elif phase == 'descend':
                    idx = np.arange(descend_idx[0],descend_idx[1])    
                else:
                    idx = np.arange(0,len(df))  
                            

                # General data
                Leg = len(df)/3600.0 # En h

                #consumption is the integral (sum) of consumption rate
                total_weight_engine1 = df['Q_1 [lb/h]'].sum()/(3600*2.205) #(to kg) 
                total_weight_engine2 = df['Q_2 [lb/h]'].sum()/(3600*2.205) #(to kg)
                total_weight = total_weight_engine1 + total_weight_engine2
                
                T_oil_init = (df['T_OIL_1 [deg C]'][0]+df['T_OIL_2 [deg C]'][0])

                # Phase data
                df_phase = df.iloc[idx]

                weight_engine1 = df_phase['Q_1 [lb/h]'].sum()/(3600*2.205) #(to kg) 
                weight_engine2 = df_phase['Q_2 [lb/h]'].sum()/(3600*2.205) #(to kg)
                volume1 = weight_engine1 /0.73 
                volume2 = weight_engine2 /0.73 
                
                phase_duration = len(df_phase)/3600 # in hour     
                average_egt = (np.mean(df_phase['EGT_1 [deg C]']) + np.mean(df_phase['EGT_2 [deg C]']))/2.0
                TAT_max = df_phase['TAT [deg C]'].max()
                TAT_min = df_phase['TAT [deg C]'].min() 
                T_oil_range_1 =  df_phase['T_OIL_1 [deg C]'].max() - df_phase['T_OIL_1 [deg C]'].min()
                T_oil_range_2 =  df_phase['T_OIL_2 [deg C]'].max() - df_phase['T_OIL_2 [deg C]'].min()
                Mach_max = df_phase["M [Mach]"].max()
                try:
                    slope = (df_phase['ALT [ft]'].max() - df_phase['ALT [ft]'].min())/len(df_phase)
                except: 
                    slope = 0
                
                #TLA
                tla_1 = df_phase['TLA_1 [deg]'].unique().shape[0]
                tla_2 = df_phase['TLA_2 [deg]'].unique().shape[0]
                # NAI
                naiv_1 = df_phase['NAIV_2 [bool]'].sum()
                naiv_2 = df_phase['NAIV_2 [bool]'].sum()

                dat+= [['AC'+ac.storename[-5:-3], 'Left', i, phase, total_weight, Leg, T_oil_init,
                         phase_duration, Alt_max, Mach_max, slope, average_egt, TAT_max, TAT_min, T_oil_range_1, tla_1, naiv_1,
                         volume1]]
                dat+= [['AC'+ac.storename[-5:-3], 'Right', i, phase, total_weight, Leg, T_oil_init,
                        phase_duration, Alt_max, Mach_max, slope, average_egt, TAT_max, TAT_min,
                        T_oil_range_2, tla_2, naiv_2,
                        volume2]]
                
    dataframe = pd.DataFrame(dat,columns = ['AC', 'ENG', 'Flight', 'Phase', 'Weight [Kg]',
                                            'Leg [h]', 'T_oil_init [C]', 'Duration [h]', 
                                            'Alt_max [ft]', 'Mach_max', 'Slope [ft/s]', 
                                            'Egt_mean [C]', 'TAT_max [C]', 'TAT_min [C]',
                                            'T_oil_range [C]', 'TLA [#]', 'NAIV [s]', 
                                            'Volume [l]'])
    
    return dataframe

In [49]:
R = []
df = pd.DataFrame()
for acfile in datafiles:
    print(f'File {acfile}:')
    ac = Opset(acfile, sortkey=sorting)
    for phase in phaselist:
        df = get_consumption_local(ac, phase=phase)
        R += [df]
df = pd.concat(R)

File ../data/Aircraft_01.h5:
  phase climb
  Record is incomplete   flight 438
  phase cruise
  Record is incomplete   flight 438
  phase descend
  Record is incomplete   flight 438
File ../data/Aircraft_02.h5:
  phase climb
  Phase descend is not found   flight 411
  Record is incomplete   flight 560
  Record is incomplete   flight 866
  Record is incomplete   flight 902
  phase cruise
  Phase descend is not found   flight 411
  Record is incomplete   flight 560
  Record is incomplete   flight 866
  Record is incomplete   flight 902
  phase descend
  Phase descend is not found   flight 411
  Record is incomplete   flight 560
  Record is incomplete   flight 866
  Record is incomplete   flight 902
File ../data/Aircraft_03.h5:
  phase climb
  Record is incomplete   flight 560
  phase cruise
  Record is incomplete   flight 560
  phase descend
  Record is incomplete   flight 560


In [50]:
df

,AC,ENG,Flight,Phase,Weight [Kg],Leg [h],T_oil_init [C],Duration [h],Alt_max [ft],Mach_max,Slope [ft/s],Egt_mean [C],TAT_max [C],TAT_min [C],T_oil_range [C],TLA [#],NAIV [s],Volume [l]
0,AC01,Left,0,climb,3956.918246,2.063611,73.074434,0.276389,39439.264501,0.616248,37.642997,797.846132,28.110368,-18.379856,13.334751,50,0.000000,824.614809
1,AC01,Right,0,climb,3956.918246,2.063611,73.074434,0.276389,39439.264501,0.616248,37.642997,797.846132,28.110368,-18.379856,15.468311,49,0.000000,825.452162
2,AC01,Left,1,climb,4040.301713,2.071944,105.611225,0.287500,39457.811063,0.640898,36.044840,809.162879,40.543801,-11.622556,15.468311,10,0.000000,874.681045
3,AC01,Right,1,climb,4040.301713,2.071944,105.611225,0.287500,39457.811063,0.640898,36.044840,809.162879,40.543801,-11.622556,16.535091,9,0.000000,870.679693
4,AC01,Left,2,climb,3667.201361,1.956111,73.074434,0.311944,40825.620025,0.624464,34.549785,788.444108,25.948032,-21.353068,22.935771,65,0.000000,896.926225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993,AC03,Right,999,descend,2690.535261,1.458611,55.472563,0.352222,37621.701406,0.583381,27.702830,495.593485,14.325476,-28.921244,16.001701,144,506.049591,278.170176
1994,AC03,Left,1000,descend,3404.696804,1.976944,55.472563,0.849167,36471.814551,0.632681,11.387601,509.650415,8.919636,-23.245112,16.001701,222,681.264939,803.358004
1995,AC03,Right,1000,descend,3404.696804,1.976944,55.472563,0.849167,36471.814551,0.632681,11.387601,509.650415,8.919636,-23.245112,16.535091,232,681.264939,831.856418
1996,AC03,Left,1001,descend,2680.301984,1.505000,48.538492,0.373889,37134.854149,0.591598,25.752990,483.587175,10.811680,-27.569784,16.535091,205,0.000000,278.907346


In [51]:
df.to_hdf(os.path.join(datadir,"out","Results.h5"),"phase_table")

In [53]:
sum(df["Duration [h]"]==0)

0

Les données sont désormais accessibles depuis ce fichier de résultats.